## Implementation of Naive Spiral with interface

In [2]:
import numpy as np
import h3
import folium
import itertools
from typing import Tuple

In [3]:
from abc import ABC, abstractmethod

# Define an interface using an abstract base class
class PathFinder(ABC):

    def __init__(self):
        pass

    @abstractmethod
    def find_next_step(self, current_position: Tuple[int, int], prob_map: np.ndarray) -> Tuple[int, int]:
        """
        Find the next step and waypoints to follow.

        Args:
            current_position (Tuple[int, int]): Current coordinates (x, y) lat, long
            prob_map: np.ndarray - of size 7*..7 equal to size of dimension
        Returns:
            Tuple[int, int] - Next step coordinates in long lat (x, y).
        """
        pass

In [91]:
class OutwardSpiralPathFinder(PathFinder):
    def __init__(self):
        self.res = 15
        self.center_hexagon = None
        self.segment_start_ij_coord = None
        self.next_path_segment = []
        self.k_ring = 1

    # Create path over an entire circle
    def ring_edge_traversal(self, repetitions, current_ij_coord, i_increment, j_increment):
        for i in range(repetitions):
            current_ij_coord = (current_ij_coord[0]+i_increment, current_ij_coord[1]+j_increment)
            self.next_path_segment.append(current_ij_coord)
        return current_ij_coord

    # Implementation of abstract method that returns next waypoint
    def find_next_step(self, current_position: Tuple[int, int], prob_map: np.ndarray) -> Tuple[int, int]:
        
        if self.center_hexagon == None:
            center_hexagon = h3.geo_to_h3(current_position[0],current_position[1],resolution=self.res)
            self.center_hexagon = center_hexagon
            center_ij_coord = h3.experimental_h3_to_local_ij(self.center_hexagon,self.center_hexagon)
    
            self.next_path_segment.append(center_ij_coord)
            self.segment_start_ij_coord = center_ij_coord
        
        current_position_ij = h3.experimental_h3_to_local_ij(self.center_hexagon,h3.geo_to_h3(current_position[0], current_position[1], resolution = self.res))
        # Waypoints are calculated based on ring
        if len(self.next_path_segment) == 1 and self.segment_start_ij_coord == current_position_ij:
            self.segment_start_ij_coord = self.ring_edge_traversal(1, self.segment_start_ij_coord, 0, -1)
            self.segment_start_ij_coord = self.ring_edge_traversal(self.k_ring-1, self.segment_start_ij_coord, 1, 0)
            self.segment_start_ij_coord = self.ring_edge_traversal(self.k_ring, self.segment_start_ij_coord, 1, 1)
            self.segment_start_ij_coord = self.ring_edge_traversal(self.k_ring, self.segment_start_ij_coord, 0, 1)
            self.segment_start_ij_coord = self.ring_edge_traversal(self.k_ring, self.segment_start_ij_coord, -1, 0)
            self.segment_start_ij_coord = self.ring_edge_traversal(self.k_ring, self.segment_start_ij_coord, -1, -1)
            self.segment_start_ij_coord = self.ring_edge_traversal(self.k_ring, self.segment_start_ij_coord, 0, -1)
            self.k_ring += 1

        if current_position_ij == self.next_path_segment[0]:
            self.next_path_segment.pop(0)
            return h3.h3_to_geo(h3.experimental_local_ij_to_h3(self.center_hexagon, self.next_path_segment[0][0], self.next_path_segment[0][1]))
        else:
            print("Previous waypoint may not be correct")
            return None

In [97]:
outward_spiral_path_finder = OutwardSpiralPathFinder()
centre = (1.340640225800291, 103.962424829341)
fake_map = np.zeros((7,7,7,7))


output = {}
waypoint = centre
steps = 200
for i in range(steps):
    waypoint = outward_spiral_path_finder.find_next_step(waypoint, fake_map)
    output[h3.geo_to_h3(waypoint[0],waypoint[1],resolution = 15)] = (steps-i)/steps

In [98]:
m = folium.Map(location=[centre[0],centre[1]], zoom_start=23, max_zoom=25)

In [99]:
# Return a colour hex value from a value between 0-1
def gradient_color(value):
    if value < 0:
        value = 0
    elif value > 1:
        value = 1

    r = int(255 * value)
    b = int(255 * value)
    g = int(255 * value)

    hex_string = '#{:02x}{:02x}{:02x}'.format(r, b, g)
    return hex_string

# Add H3 hexagons as polygons to the map, m
def add_hex_to_map(hexagon_values, m):
    keys = list(hexagon_values.keys())
    num_keys = len(keys)

    for i, hexagon_id in enumerate(hexagon_values):
        vertices = h3.h3_to_geo_boundary(hexagon_id)
        color = color = gradient_color(hexagon_values[hexagon_id])
        folium.Polygon(locations=vertices, color=color, fill=True, fill_opacity=0.6).add_to(m)
        folium.map.Marker(h3.h3_to_geo(hexagon_id),
                icon=folium.DivIcon(
                    icon_size=(10,10),
                    icon_anchor=(5,14),
                    html=f'<div style="font-size: 8pt">{i}</div>'
                )
                ).add_to(m)

In [100]:
add_hex_to_map(output, m)

In [101]:
display(m)

## Implementation of Naive Path Algorithm
1. Find the long lat boundaries -> convert the indexes to ij coordinates
2. Create a set of index arrays that represent the traversal path
3. Convert that array into actual hex indices
4. Return them
5. Visualise them on the graph

In [1]:
import h3
import folium

In [2]:
# SUTD Campus
# tl - top left
tl_long_lat = (1.3410324284697368, 103.96203617658846)
tr_long_lat = (1.3410324284697368, 103.96297579829188)
bl_long_lat = (1.3399326845784334, 103.96203617658846)
br_long_lat = (1.3399326845784334, 103.96297579829188)

In [3]:
# Return a colour hex value from a value between 0-1
def gradient_color(value):
    if value < 0:
        value = 0
    elif value > 1:
        value = 1

    r = int(255 * value)
    b = int(255 * value)
    g = int(255 * value)

    hex_string = '#{:02x}{:02x}{:02x}'.format(r, b, g)
    return hex_string

In [4]:
# Add H3 hexagons as polygons to the map, m
def add_hex_to_map(hexagon_values, m):
    keys = list(hexagon_values.keys())
    num_keys = len(keys)

    for i, hexagon_id in enumerate(hexagon_values):
        vertices = h3.h3_to_geo_boundary(hexagon_id)
        color = color = gradient_color(hexagon_values[hexagon_id])
        folium.Polygon(locations=vertices, color=color, fill=True, fill_opacity=0.6).add_to(m)
        folium.map.Marker(h3.h3_to_geo(hexagon_id),
                icon=folium.DivIcon(
                    icon_size=(10,10),
                    icon_anchor=(5,14),
                    html=f'<div style="font-size: 8pt">{i}</div>'
                )
                ).add_to(m)

In [13]:
res = 12
m = folium.Map(location=[1.3410324284697368, 103.96203617658846], zoom_start=20)

In [6]:
# Hexagon index
tl_hexagon = h3.geo_to_h3(tl_long_lat[0],tl_long_lat[1],resolution=res)
tr_hexagon = h3.geo_to_h3(tr_long_lat[0],tr_long_lat[1],resolution=res)
bl_hexagon = h3.geo_to_h3(bl_long_lat[0],bl_long_lat[1],resolution=res)
br_hexagon = h3.geo_to_h3(br_long_lat[0],br_long_lat[1],resolution=res)

In [7]:
# Convert the hexagon index into i-j coordinates
tl_ij_coord = h3.experimental_h3_to_local_ij(tl_hexagon,tl_hexagon)
tr_ij_coord = h3.experimental_h3_to_local_ij(tl_hexagon,tr_hexagon)
bl_ij_coord = h3.experimental_h3_to_local_ij(tl_hexagon,bl_hexagon)
br_ij_coord = h3.experimental_h3_to_local_ij(tl_hexagon,br_hexagon)

print(tl_ij_coord, tr_ij_coord, bl_ij_coord, br_ij_coord)

def vector_edge(coord_a, coord_b):
    return (coord_a[0]-coord_b[0],coord_a[1]-coord_b[1])

print("left_edge_vector:", vector_edge(tl_ij_coord,bl_ij_coord))
print("right_edge_vector:", vector_edge(tr_ij_coord,br_ij_coord))
print("top_edge_vector:", vector_edge(tr_ij_coord,tl_ij_coord))
print("btm_edge_vector:", vector_edge(br_ij_coord,bl_ij_coord))

(70855, 21262) (70860, 21268) (70859, 21259) (70864, 21265)
left_edge_vector: (-4, 3)
right_edge_vector: (-4, 3)
top_edge_vector: (5, 6)
btm_edge_vector: (5, 6)


In [8]:
# Creating a naive path
def naive_path_generator(tl_ij_coord, tr_ij_coord, bl_ij_coord, br_ij_coord):
    left_edge_hex_line = h3.h3_line(tl_hexagon,bl_hexagon)
    right_edge_hex_line = h3.h3_line(tr_hexagon,br_hexagon)
    path =[]
    is_return = False
    for left_hex, right_hex in zip(left_edge_hex_line, right_edge_hex_line):
        if is_return:
            left_hex, right_hex = right_hex, left_hex
        path += h3.h3_line(left_hex,right_hex)
        is_return = not is_return
    return path

In [9]:
hex_path = naive_path_generator(tl_ij_coord, tr_ij_coord, bl_ij_coord, br_ij_coord)

In [10]:
def norm_dict_from_ls(input_list):
    if not input_list:
        return {}

    list_length = len(input_list)
    result_dict = {}

    for index, value in enumerate(input_list):
        normalized_index = index / list_length
        result_dict[value] = normalized_index

    return result_dict

In [14]:
hex_line_d = norm_dict_from_ls(hex_path)
print(hex_line_d)

add_hex_to_map(hex_line_d, m)
display(m)

{'8c6526ac342e7ff': 0.0, '8c6526ac342e5ff': 0.017857142857142856, '8c6526ac34233ff': 0.03571428571428571, '8c6526ac34231ff': 0.05357142857142857, '8c6526ac34235ff': 0.07142857142857142, '8c6526ac34223ff': 0.08928571428571429, '8c6526ac34221ff': 0.10714285714285714, '8c6526ac34229ff': 0.125, '8c6526ac3422bff': 0.14285714285714285, '8c6526ac3423dff': 0.16071428571428573, '8c6526ac34239ff': 0.32142857142857145, '8c6526ac3423bff': 0.19642857142857142, '8c6526ac342edff': 0.21428571428571427, '8c6526ac342e1ff': 0.23214285714285715, '8c6526ac342ebff': 0.25, '8c6526ac342e9ff': 0.26785714285714285, '8c6526ac34217ff': 0.2857142857142857, '8c6526ac34215ff': 0.30357142857142855, '8c6526ac34207ff': 0.3392857142857143, '8c6526ac34205ff': 0.35714285714285715, '8c6526ac3420dff': 0.375, '8c6526ac34201ff': 0.39285714285714285, '8c6526ac34203ff': 0.4107142857142857, '8c6526ac3421dff': 0.5714285714285714, '8c6526ac34211ff': 0.44642857142857145, '8c6526ac34213ff': 0.4642857142857143, '8c6526ac342c5ff': 0.4

In [16]:
m.save('../output/sutd_map.html')